# 1. Installing And Import Dependencies

In [106]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!pip install transformers streamlit requests beautifulsoup4 pandas numpy 

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bs4 import BeautifulSoup
import torch
import requests
import re
import numpy as np
import pandas as pd

# 2. Creating the Model

In [57]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


# 3. Training and Testing the Model

In [102]:
tokens = tokenizer.encode('je l ai absolument détesté. ne réessayez jamais',return_tensors='pt')

In [71]:
tokenizer.decode(tokens[0])

'[CLS] je l ai absolument deteste. ne reessayez jamais [SEP]'

In [103]:
result = model(tokens)

In [99]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.3854,  2.5690,  1.7249, -1.5135, -3.1884]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [104]:
result.logits

tensor([[ 4.4548,  2.0282, -0.4900, -2.7767, -2.3229]],
       grad_fn=<AddmmBackward0>)

In [105]:
int(torch.argmax(result.logits))+1

1

# 4. Collecting Reviews

In [18]:
r = requests.get('https://www.yelp.com/biz/cuisine-of-nepal-san-francisco')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})
reviews =[result.text for result in results] 

In [19]:
reviews[1]

'I took my wife here for her birthday dinner, and we enjoyed everything we order. We arrive for an early meal at 5:45pm, and there was ample seating. We were seated, we ordered, and the food came as soon as the rest of the patrons to this spot. This place filled up quickly for the dinner rush!The service was attentive, friendly, and quick. We ordered the:Veggie Momos which were filled with sautéed greens, cilantro, onions, carrots, and delicious herbs. The sauce was also delicious, and added a nice warmth and spice to the momos. Palak Paneer was creamy with a hint of herbs, and the paneer cubes were very rich and milky tasting. Machhako Tarkari Curry was a delicious salmon curry. It was a perfect compliment to the creamy palak planner, as it was a lighter more tomato based sauce. We also enjoyed that the entree dishes come with a side of rice and naan!For dessert we order Lal Mohan, yummy soft pudding like donut holes! They were so cute in the little bowl they came out in too. We will 

# 5. Loading the reviews into a DataFrame and Scoring

In [20]:
df = pd.DataFrame(np.array(reviews), columns=['Reviews'])

In [21]:
df

,Reviews
0,"When I got my meal delivered by Doordash, it w..."
1,"I took my wife here for her birthday dinner, a..."
2,We were craving hearty comfort food on a rainy...
3,"We came here to try the Salmon Curry, vegetari..."
4,My cousin recommended this place. Tommy surpri...
5,When our restaurant reservation fell though be...
6,I can not thank Mr Shrestha enough for his hos...
7,Fantastic experience on all levels - this was ...
8,Nice spot for some unique cuisine. The flavors...
9,"We have been going here for years, but I've ne..."


In [22]:
df['Reviews'].iloc[0]

"When I got my meal delivered by Doordash, it was absolutely delicious! \xa0The soup and the chicken meal i ordered was cooked perfectly and very tasty. \xa0It was a time of struggle too and was so happy that even when times are tough, i was still able to enjoy a very tasteful meal by this restaurant delivered by Doordash. \xa0Thats why I am so thankful that I was even able to get a free meal from doordash because of their special offer this month. \xa0I honestly dont want to keep it from anyone because I feel like someone else like myself can also feel the joy of getting a few meals delivered without even having to pay a dime for. \xa0Thank you so much DD! \xa0And for anyone who wants this same offer, you can get it here. \xa0Just fill out the form all the way through and you can also get 1 week's food for free: \xa0tpmbdirect.com/1600/60/y…"

In [23]:
def sentiment_score(review):
    tokens = tokenizer.encode(review,return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [24]:
sentiment_score(df['Reviews'].iloc[0])

5

In [25]:
df['Sentiment'] = df['Reviews'].apply(lambda x: sentiment_score(x[:512]))

In [26]:
df

,Reviews,Sentiment
0,"When I got my meal delivered by Doordash, it w...",5
1,"I took my wife here for her birthday dinner, a...",5
2,We were craving hearty comfort food on a rainy...,3
3,"We came here to try the Salmon Curry, vegetari...",5
4,My cousin recommended this place. Tommy surpri...,5
5,When our restaurant reservation fell though be...,4
6,I can not thank Mr Shrestha enough for his hos...,4
7,Fantastic experience on all levels - this was ...,5
8,Nice spot for some unique cuisine. The flavors...,4
9,"We have been going here for years, but I've ne...",2


In [30]:
df['Reviews'].iloc[1]

'I took my wife here for her birthday dinner, and we enjoyed everything we order. We arrive for an early meal at 5:45pm, and there was ample seating. We were seated, we ordered, and the food came as soon as the rest of the patrons to this spot. This place filled up quickly for the dinner rush!The service was attentive, friendly, and quick. We ordered the:Veggie Momos which were filled with sautéed greens, cilantro, onions, carrots, and delicious herbs. The sauce was also delicious, and added a nice warmth and spice to the momos. Palak Paneer was creamy with a hint of herbs, and the paneer cubes were very rich and milky tasting. Machhako Tarkari Curry was a delicious salmon curry. It was a perfect compliment to the creamy palak planner, as it was a lighter more tomato based sauce. We also enjoyed that the entree dishes come with a side of rice and naan!For dessert we order Lal Mohan, yummy soft pudding like donut holes! They were so cute in the little bowl they came out in too. We will 

In [29]:
sentiment_score(df['Reviews'].iloc[1])

5